In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('crypto.csv', dtype=object)

In [ ]:
df.head()

,Unnamed: 0,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,quoted_status.place.id,quoted_status.place.url,quoted_status.place.place_type,quoted_status.place.name,quoted_status.place.full_name,quoted_status.place.country_code,quoted_status.place.country,quoted_status.place.contained_within,quoted_status.place.bounding_box.type,quoted_status.place.bounding_box.coordinates
0,0,Wed May 18 15:10:30 +0000 2022,1526943314691477505,1526943314691477505,RT @MLGDan23: NFT NYC!!! LQG!!!! @TribeQuokka ...,False,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Wed May 18 15:09:40 +0000 2022,1526943108339888128,1526943108339888128,sophie_millerst https://t.co/zzJScjs5ZA 18+ #B...,False,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Wed May 18 15:07:10 +0000 2022,1526942477499789312,1526942477499789312,Mit #Lending von #Axie Infinity (#AXS) auf #ku...,True,"<a href=""http://mk85.de"" rel=""nofollow"">John W...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed May 18 15:04:56 +0000 2022,1526941915471544321,1526941915471544321,RT @chris40004: Neue wöchentliche Umfrage mit ...,False,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Wed May 18 15:02:11 +0000 2022,1526941224560742400,1526941224560742400,RT @dogefathereco: Auch... 😢\n\n#memecoin #lun...,False,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(300, 325)

In [ ]:
#list(df.columns)

Nodes:

        User nodes – represents a Twitter user (handle and number of followers)
        Tweet nodes – represent a tweet (text, number of likes)
        Hashtag nodes – represent a hashtag
        Country nodes – represent a country (country name, country code)
        Device nodes – represent a device

Relationships:

        TWEETED relationship – in between a User and a Tweet; indicates that this user is the author of the tweet; also indicates the date at which it was tweeted
        RETWEETED relationship – in between a User and a User; indicates this user retweeted this tweet; also indicates the date at which it was retweeted
        USED_HASHTAG relationship – in between a User and a Hashtag
        MENTIONED relationship – in between two Users
        FROM relation – in between a User and a Country
        USED relationship - in between a User and a Source Device


In [5]:
df['retweeted_status.user.id_str'].dtypes

dtype('O')

Extract Hashtags

In [6]:
import json
hashtags = []
for i in range(len(df)):
    tmps = json.loads(df['entities.hashtags'][i].replace("'",'"'))
    for c in range(len(tmps)):
        hashtags.append(tmps[c]['text'])
hashtags = set(hashtags)

Extract User

In [7]:
user_col = [col for col in df if col.startswith('user.')]
user = df[user_col].copy()
user.drop_duplicates(inplace=True)

In [8]:
quoted_user_col = [col for col in df if col.startswith('quoted_status.user.')]
quoted_user = df[quoted_user_col].copy()
quoted_user.drop_duplicates(inplace=True)
quoted_user.columns=quoted_user.columns.str.replace("quoted_status.", "")

/tmp/ipykernel_94208/3256792357.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  quoted_user.columns=quoted_user.columns.str.replace("quoted_status.", "")


In [9]:
retweeted_user_col = [col for col in df if col.startswith('retweeted_status.user.')]
retweeted_user = df[retweeted_user_col].copy()
retweeted_user.drop_duplicates(inplace=True)
retweeted_user.columns=retweeted_user.columns.str.replace("retweeted_status.", "")

/tmp/ipykernel_94208/557703801.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  retweeted_user.columns=retweeted_user.columns.str.replace("retweeted_status.", "")


Sources

In [11]:
import re
df['source_clean'] = df['source'].apply(lambda x: re.sub('<[^<]+?>', '', x))
sources = set(df['source_clean'])

Locations

Create Nodes

In [12]:
from py2neo import Graph, Node, Relationship, NodeMatcher
graph = Graph("http://localhost:7474", password="1710")
graph.delete_all()

In [13]:
for line in sources:
    node = Node("Device", source=line)
    graph.create(node)

In [14]:
for line in hashtags:
    node = Node("Hashtag", hashtag=line)
    graph.create(node)

User Nodes

In [15]:
df_all_user = pd.concat([user, quoted_user, retweeted_user]).drop_duplicates(subset=['user.id'])

In [16]:
locations = set(df_all_user['user.location'].fillna('na'))
for line in locations:
    node = Node("Location", location=line)
    graph.create(node)

In [17]:
matcher = NodeMatcher(graph)
for i in range(len(df_all_user)):
    line = df_all_user.iloc[i].fillna('na')
    id = line['user.id_str']
    name = line['user.screen_name']
    created = line['user.created_at']
    followers = line['user.followers_count']
    node = Node("User", id=id, name=name, created=created, followers=followers)
    node.add_label("User")
    graph.create(node)
    # originates from
    if line['user.location'] != 'na':
        location = matcher.match("Location", location=line['user.location']).first()
        rel = Relationship(node, "originates_from", location)
        graph.create(rel)

Tweet Nodes

In [18]:
for i in range(len(df)):
    line = df.iloc[i].fillna('na')
    id = line['id_str']
    text = line['text']
    created = line['created_at']
    node = Node("Tweet", id=id, text=text, created=created)
    graph.create(node)
    # tweeted relationship
    user_node = matcher.match("User", id = line['user.id_str']).first()
    rel = Relationship(user_node, "tweeted", node)
    graph.create(rel)
    # uses device
    device = matcher.match("Device", source=line['source_clean']).first()
    rel = Relationship(user_node, "uses", device)
    graph.create(rel)
    # retweeted relationship
    if line['retweeted_status.user.id_str'] != 'na':
        retweeted_user = matcher.match("User", id=line['retweeted_status.user.id_str']).first()
        rel = Relationship(user_node, "retweeted", retweeted_user)
        graph.create(rel)
    # quoted relationship
    if line['quoted_status.user.id_str'] != 'na':
        quoted_user = matcher.match("User", id=line['quoted_status.user.id_str']).first()
        rel = Relationship(user_node, "quoted", quoted_user)
        graph.create(rel)
    # used_hashtag
    tmps = json.loads(line['entities.hashtags'].replace("'",'"'))
    for tmp in tmps:
        hashtag = matcher.match("Hashtag", hashtag=tmp['text']).first()
        rel = Relationship(user_node, "used_hashtag", hashtag)
        graph.create(rel)
    # mentions
    try:
        tmps = json.loads(line['entities.user_mentions'].replace("'",'"'))
        for tmp in tmps:
            user = matcher.match("User", id=tmp['id_str']).first()
            if not user:
                user = Node("User", id=tmp['id_str'], name=tmp['screen_name'])
                graph.create(user)
            rel = Relationship(user_node, "mentioned", user)
            graph.create(rel)
    except:
        print(f"Cannot parse {line['entities.user_mentions']}")

 

Cannot parse [{'screen_name': 'ZedOne16', 'name': "Camper's Diary", 'id': 1054992154039386112, 'id_str': '1054992154039386112', 'indices': [3, 12]}]


In [19]:
import os
os.getcwd()

'/home/pierre/fakenews'

In [23]:
files =[file for file in os.listdir("../") if '.csv' in file]
files

['test.csv', 'crypto.csv', 'ukraine.csv', 'impfung.csv', '_apple.csv']